In [20]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 크롤링할 페이지 URL
url = 'https://www.evpost.co.kr/wp/category/%ec%8b%a4%ec%82%a4%ec%9a%a9%ea%b8%b0/%ec%b9%bc%eb%9f%bc/'

# 웹페이지의 HTML 데이터를 가져옴
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 기사 목록 추출
articles = soup.find_all('div', class_='td-module-thumb')

# 스크랩한 데이터를 저장할 리스트
news_data = []

# 기사 제목, 링크, 본문 미리보기 추출
for article in articles:
    title = article.find('a')['title']  # 제목
    link = article.find('a')['href']  # 기사 링크
    
    # 기사 본문 페이지로 이동해 본문 내용 크롤링
    article_response = requests.get(link)
    article_soup = BeautifulSoup(article_response.text, 'html.parser')
    
    # 본문에서 첫 70자 추출
    content_tag = article_soup.find('div', class_='td-post-content')  # 본문 내용이 있는 div 태그
    if content_tag:
        content = content_tag.get_text(strip=True)  # 본문 텍스트를 추출하고 공백 제거
        if content:  # 본문 내용이 공백이 아니면 처리
            preview_text = content[:70]  # 첫 70자를 미리보기로 사용
            
            # 수집한 데이터를 리스트에 추가 (본문이 있을 때만)
            news_data.append({
                '제목': title,
                '링크': link,
                '미리보기': preview_text
            })
    else:
        print(f"본문을 찾을 수 없는 기사: {title} (제외됨)")

# 결과를 pandas DataFrame으로 변환
df = pd.DataFrame(news_data)

# 데이터프레임이 비어있는지 확인
if df.empty:
    print("데이터프레임이 비어 있습니다. 크롤링한 데이터가 없습니다.")
else:
    print("수집된 데이터 확인:")
    print(df.head())
    
# 저장할 파일 경로 설정 (app_manager/data 폴더 내에 저장)
data_folder = os.path.join('app_manager', 'data')  # app_manager/data 폴더 경로
os.makedirs(data_folder, exist_ok=True)  # 폴더가 없으면 생성

# 저장할 CSV 파일 경로 설정
csv_file_path = os.path.join(data_folder, 'evpost_news.csv')

# 데이터프레임을 CSV 파일로 저장
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# CSV 파일이 제대로 저장되었는지 확인
if os.path.exists(csv_file_path):
    print(f"CSV 파일이 성공적으로 저장되었습니다: {csv_file_path}")

    # 저장된 CSV 파일을 다시 읽어서 확인
    df_loaded = pd.read_csv(csv_file_path, encoding='utf-8-sig')
    print("CSV 파일 로드 성공")
    print(df_loaded.head())
else:
    print(f"CSV 파일을 저장할 수 없습니다: {csv_file_path}")


수집된 데이터 확인:
                                 제목  \
0         캐스퍼 일렉트릭, 딱!3290만원 주고 사세요   
1  ‘EV3 깡통도 풀옵션 수준이라던데..'(feat.라펙터)   
2          테슬라 에너지, 이래서 잘될 수밖에 없습니다   
3                      이러면 완전히 나가린데   
4           디자인 맛집이 말아주는 전기차 ft.아우디   

                                                  링크  \
0  https://www.evpost.co.kr/wp/%ec%ba%90%ec%8a%a4...   
1  https://www.evpost.co.kr/wp/ev3-%ea%b9%a1%ed%8...   
2  https://www.evpost.co.kr/wp/%ed%85%8c%ec%8a%ac...   
3  https://www.evpost.co.kr/wp/%ec%9d%b4%eb%9f%ac...   
4  https://www.evpost.co.kr/wp/%eb%94%94%ec%9e%90...   

                                                미리보기  
0  국내 전기차 대중화를 이끌 현대자동차의 캐스퍼 일렉트릭, 기아의 EV3가 연이어 출...  
1  EV3와 캐스퍼 일렉트릭이 연이어 출시됐습니다. 가성비를 앞세웠지만 첨단기술을 탑재...  
2  테슬라 에너지 사업의 핵심 경쟁력[요약]1. 재생 에너지 보급은 (1) 짧은 건설 ...  
3  지난주 부산에서 캐스퍼 일렉트릭이 공개됐습니다. 두 달 전쯤 캐스퍼를 주문할 때 캐...  
4  신차 미리 보기의 여섯 번째 신미(新美)는 더 뉴 아우디 e-트론 GT입니다. 토니...  
CSV 파일이 성공적으로 저장되었습니다: app_manager\data\evpost_news.csv
CSV 파일 로드 성공
                 